# Use Pair RDDs to join two datasets

In [4]:
from pyspark import SparkContext
sc = SparkContext()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.storage.StorageUtils$
	at org.apache.spark.storage.BlockManagerMasterEndpoint.<init>(BlockManagerMasterEndpoint.scala:110)
	at org.apache.spark.SparkEnv$.$anonfun$create$9(SparkEnv.scala:348)
	at org.apache.spark.SparkEnv$.registerOrLookupEndpoint$1(SparkEnv.scala:287)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:336)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:191)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:277)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:460)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:67)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java

## Step 1. Load and parse data data

In [3]:
# create an RDD from the data file
logfile = "/loudacre/weblogs/201*.log"
logs = sc.textFile("hdfs://localhost:9000/"+logfile)

NameError: name 'sc' is not defined

In [9]:
# Filter : create an RDD containing only those lines that are requests for JPG files
jpglogs = logs.filter(lambda x: ".jpg" in x)

for line in jpglogs.take(5):
    print(line)

217.150.149.167 - 4712 [15/Sep/2013:23:56:06 +0100] "GET /ronin_s4.jpg HTTP/1.0" 200 5552 "http://www.loudacre.com"  "Loudacre Mobile Browser MeeToo 1.0"
104.184.210.93 - 28402 [15/Sep/2013:23:42:53 +0100] "GET /titanic_2200.jpg HTTP/1.0" 200 19466 "http://www.loudacre.com"  "Loudacre Mobile Browser MeeToo 2.0"
37.91.137.134 - 36171 [15/Sep/2013:23:39:33 +0100] "GET /ronin_novelty_note_3.jpg HTTP/1.0" 200 7432 "http://www.loudacre.com"  "Loudacre Mobile Browser iFruit 3"
177.43.223.203 - 90653 [15/Sep/2013:23:31:17 +0100] "GET /ifruit_3.jpg HTTP/1.0" 200 19578 "http://www.loudacre.com"  "Loudacre Mobile Browser Sorrento F31L"
19.250.65.76 - 44388 [15/Sep/2013:23:31:10 +0100] "GET /sorrento_f24l.jpg HTTP/1.0" 200 5730 "http://www.loudacre.com"  "Loudacre Mobile Browser iFruit 3A"


In [11]:
lengths = logs.map(lambda s: len(s))
lengths.take(5)

[151, 143, 154, 147, 160]

In [12]:
logwords = logs.map(lambda line: line.split(" "))
logwords.take(1)

[['3.94.78.5',
  '-',
  '69827',
  '[15/Sep/2013:23:58:36',
  '+0100]',
  '"GET',
  '/KBDOC-00033.html',
  'HTTP/1.0"',
  '200',
  '14417',
  '"http://www.loudacre.com"',
  '',
  '"Loudacre',
  'Mobile',
  'Browser',
  'iFruit',
  '1"']]

In [14]:
## Get all IPs 

ips = logs.map(lambda line: line.split(" ")[0])
# print the first 5 IP address
for line in ips.take(5):
    print(line)

3.94.78.5
3.94.78.5
19.38.140.62
19.38.140.62
129.133.56.105


In [15]:
ips.saveAsTextFile("/loudacre/iplist")

## Step 2. Using map-reduce to count the number of requests from each user

In [17]:
userCount = logs.map(lambda row:(row[1],1)).reduceByKey(lambda a,b:a+b)

In [18]:
# use type function to verify the type of the variable
type(userCount)

pyspark.rdd.PipelinedRDD

In [19]:
# print the outcome in formatted fasion
for row in userCount.take(5):
    print("{},{}".format(*row))  # the operator * unpack the elements in the `row` tuple

1,125136
8,82408
4,125142
2,121599
9,78175


## Step 3. Determine how many users visited the site for each frequency (histogram)

In [29]:
freqCount = userCount.map(lambda (user, cnt):(cnt,user)).countByKey()

SyntaxError: invalid syntax (Temp/ipykernel_20736/1147980971.py, line 1)

In [ ]:
type(freqCount)

In [ ]:
for item in freqCount.items():
    print("{}:{}".format(*item))

## Step 4. Build an IP address list for each user

### A. Frist build a list of <user_id, ip> pairs sorted by user id 

In [30]:
userIpSorted = logs.map(lambda row:(row[1],row[0])) \
    .sortByKey()

In [31]:
for item in userIpSorted.take(10):
    print("{}\t{}".format(*item))

.	3
.	3
.	6
.	6
.	5
.	5
.	7
.	7
.	2
.	2


### B. Create an RDD where the user id is the key, and the value is the list of all the IP addresses that user has connected from.

In [32]:
userIpList = logs.map(lambda row:(row[1],row[0])) \
    .groupByKey() \
    .mapValues(list)

In [33]:
for item in userIpList.take(5):
    print("{}\t{}".format(item[0],item[1]))

1	['2', '2', '2', '2', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '7', '7', '1', '1', '2', '2', '2', '2', '1', '1', '2', '2', '2', '2', '2', '2', '1', '1', '6', '6', '2', '2', '2', '2', '2', '2', '6', '6', '1', '1', '2', '2', '1', '1', '5', '5', '1', '1', '1', '1', '8', '8', '1', '1', '1', '1', '9', '9', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '6', '6', '6', '6', '2', '2', '2', '2', '2', '2', '1', '1', '8', '8', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '4', '4', '4', '4', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '4', '4', '8', '8', '2', '2', '2', '2', '1', '1', '3', '3', '9', '9', '5', '5', '3', '3', '3', '3', '1', '1', '1', '1', '2', '2', '8', '8', '4', '4', '2', '2', '2', '2', '5', '5', '8', '8', '2', '2', '7', '7', '4', '4', '7', '7', '1', '1', '2', '2', '2', '2', '2', '2', '8', '8', '2', '2', '2', '2', '5', '5', '1', '1

### C. Create an RDD where the user id is the key, and the value is the distinct list of all the IP addresses that user has connected from.

In [34]:
userIpListDistinct = logs.map(lambda row:(row[1],row[0])) \
    .groupByKey() \
    .mapValues(set)

In [35]:
for item in userIpListDistinct.take(5):
    print("{}\t{}".format(item[0],list(item[1])))

1	['5', '3', '2', '7', '4', '1', '8', '9', '6']
8	['5', '3', '2', '7', '4', '1', '9', '8', '6']
4	['5', '3', '2', '7', '4', '1', '8', '9', '6']
2	['5', '3', '2', '7', '4', '1', '8', '9', '6']
9	['5', '3', '2', '7', '4', '1', '8', '9', '6']


# Join Web Log Data with Account Data 

## Step 1. Load and parse data data

In [40]:
# accounts = sc.textFile("file:C:\\Users\\abisht\\accounts\\*") \
#     .map(lambda line:line.split(',')) \
#     .cache()

accfile = "/account/accounts"
accounts = sc.textFile("hdfs://localhost:9000/"+accfile)
accounts.map(lambda line:line.split(',')) \
    .cache()

PythonRDD[58] at RDD at PythonRDD.scala:53

In [41]:
accounts.take(1)

['1,2008-10-23 16:05:05.0,\\N,Donald,Becton,2275 Washburn Street,Oakland,CA,94660,5100032418,2014-03-18 13:29:47.0,2014-03-18 13:29:47.0']

## Step 2. Join the accounts data with userCount data.

In [43]:
joined = accounts.keyBy(lambda row:row[0]) \
    .join(userCount) \
    .map(lambda (userid,(row,count)):(userid, count,row[3]+' '+row[4]))

SyntaxError: invalid syntax (Temp/ipykernel_20736/4271150502.py, line 3)

In [44]:
joined.take(5)

NameError: name 'joined' is not defined

## Step 3. Create a list of names by Postal Code

In [45]:
postalRoster = accounts.keyBy(lambda row:row[8]) \
    .mapValues(lambda row:(row[3],row[4])) \
    .groupByKey() \
    .mapValues(lambda names:list(names))

In [51]:
first5postal = postalRoster.takeOrdered(5,lambda postal,names:postal)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 56.0 failed 1 times, most recent failure: Lost task 1.0 in stage 56.0 (TID 1855) (20090078-7V3L33.seattleu.edu executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\lib\pyspark.zip\pyspark\worker.py", line 609, in process
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\pyspark\rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\Users\abisht\Downloads\spark-3.2.1-bin-hadoop3.2\spark-3.2.1-bin-hadoop3.2\python\pyspark\rdd.py", line 2

In [ ]:
# Python lambda function can't unpack a tuple parameter into two arguments:

# rdd5 = rdd4.map(lambda x, y:(x, y, check(x,y,6))) 

#should be written as 

#rdd5 = rdd4.map(lambda t: (t[0], t[1], check(t[0], t[1], 6)))
# print(rdd5.take(3))

In [ ]:
for row in first5postal:
    print("--- {}".format(row[0]))
    for name in row[1]:
        print("{}, {}".format(name[1],name[0]))
--- 85000